In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
import tensorflow as tf

# Check TensorFlow GPU availability
if tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None):
    print("GPU is available for TensorFlow")
else:
    print("GPU is NOT available for TensorFlow")

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
GPU is NOT available for TensorFlow


In [4]:
df = pd.read_csv('./mov-dataset.csv')
df.head()


,X,Y,S_val,X_mov,Y_mov
0,0.649460,0.537015,0.0,0.000000,0.000000
1,0.649460,0.537015,0.0,0.000000,0.000000
2,0.649460,0.537015,0.0,0.000000,0.000000
3,0.649764,0.537047,0.0,0.000304,0.000033
4,0.649764,0.537047,0.0,0.000000,0.000000


In [5]:
X_input = df[['X','Y','S_val']]
Y_target = df[['X_mov','Y_mov']]

In [17]:
# Splitting dataset into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X_input, Y_target, test_size=0.16, random_state=42)


In [18]:
Y_train.shape

(990963, 2)

In [19]:
X_train.shape

(990963, 3)

In [20]:
Y_val.shape

(188755, 2)

In [21]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam

# Define the model architecture
def build_model():
    # Input layer
    inputs = Input(shape=(3,), name='input_layer')  # For your features X, Y, S_val

    # Hidden layers
    x = Dense(128, activation='relu', name='hidden_layer_1')(inputs)
    x = Dense(64, activation='relu', name='hidden_layer_2')(x)
    x = Dense(32, activation='relu', name='hidden_layer_3')(x)

    # Output layer
    outputs = Dense(2, activation='linear', name='output_layer')(x)  # For your targets X_mov, Y_mov

    # Create model
    model = Model(inputs=inputs, outputs=outputs, name='multi_output_regression_model')

    return model


In [23]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define checkpoint callback to save only the best model based on validation loss
model_checkpoint_callback = ModelCheckpoint(
    filepath='./best_model.keras',  # File path to save the model
    save_best_only=True,  # Only save the best model
    monitor='val_loss',  # Monitor the validation loss
    mode='min',  # 'min' means the lowest validation loss is considered the "best"
    verbose=1  # Log a message when the best model is updated/saved
)


In [ ]:
# Assuming the model has been defined and compiled as per previous instructions
model = build_model()
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='mae',  # Mean Squared Error
              metrics=['mae','mse'])  # Mean Absolute Error as a metric for monitoring

# Training the model
history = model.fit(X_train, Y_train,
                    batch_size=64,  # Adjust based on your GPU's memory
                    epochs=2000,  # Number of epochs to train for
                    validation_data=(X_val, Y_val),
                    callbacks=[model_checkpoint_callback])


Epoch 1/2000
15474/15484 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0020 - mae: 0.0020 - mse: 5.6241e-04
Epoch 1: val_loss improved from inf to 0.00183, saving model to ./best_model.keras
15484/15484 ━━━━━━━━━━━━━━━━━━━━ 24s 1ms/step - loss: 0.0020 - mae: 0.0020 - mse: 5.6240e-04 - val_loss: 0.0018 - val_mae: 0.0018 - val_mse: 6.2546e-04
Epoch 2/2000
15457/15484 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 0.0018 - mae: 0.0018 - mse: 5.7832e-04
Epoch 2: val_loss improved from 0.00183 to 0.00182, saving model to ./best_model.keras
15484/15484 ━━━━━━━━━━━━━━━━━━━━ 24s 2ms/step - loss: 0.0018 - mae: 0.0018 - mse: 5.7828e-04 - val_loss: 0.0018 - val_mae: 0.0018 - val_mse: 6.2545e-04
Epoch 3/2000
 6708/15484 ━━━━━━━━━━━━━━━━━━━━ 12s 1ms/step - loss: 0.0018 - mae: 0.0018 - mse: 5.5133e-04